<h1>CS4618: Artificial Intelligence I</h1>
<h1>Error Estimation</h1>
<h2>
    Derek Bridge<br>
    School of Computer Science and Information Technology<br>
    University College Cork
</h2>

<h1>Initialization</h1>
$\newcommand{\Set}[1]{\{#1\}}$ 
$\newcommand{\Tuple}[1]{\langle#1\rangle}$ 
$\newcommand{\v}[1]{\pmb{#1}}$ 
$\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ 
$\newcommand{\rv}[1]{[#1]}$ 
$\DeclareMathOperator{\argmax}{arg\,max}$ 
$\DeclareMathOperator{\argmin}{arg\,min}$ 
$\DeclareMathOperator{\dist}{dist}$
$\DeclareMathOperator{\abs}{abs}$

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

# Class, for use in pipelines, to select certain columns from a DataFrame and convert to a numpy array
# From A. Geron: Hands-On Machine Learning with Scikit-Learn & TensorFlow, O'Reilly, 2017
# Modified by Derek Bridge to allow for casting in the same ways as pandas.DataFrame.astype
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names, dtype=None):
        self.attribute_names = attribute_names
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_selected = X[self.attribute_names]
        if self.dtype:
            return X_selected.astype(self.dtype).values
        return X_selected.values
    
# Class, for use in pipelines, to binarize nominal-valued features (while avoiding the dummy variabe trap)
# By Derek Bridge, 2017
class FeatureBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self, features_values):
        self.features_values = features_values
        self.num_features = len(features_values)
        self.labelencodings = [LabelEncoder().fit(feature_values) for feature_values in features_values]
        self.onehotencoder = OneHotEncoder(sparse=False,
            n_values=[len(feature_values) for feature_values in features_values])
        self.last_indexes = np.cumsum([len(feature_values) - 1 for feature_values in self.features_values])
    def fit(self, X, y=None):
        for i in range(0, self.num_features):
            X[:, i] = self.labelencodings[i].transform(X[:, i])
        return self.onehotencoder.fit(X)
    def transform(self, X, y=None):
        for i in range(0, self.num_features):
            X[:, i] = self.labelencodings[i].transform(X[:, i])
        onehotencoded = self.onehotencoder.transform(X)
        return np.delete(onehotencoded, self.last_indexes, axis=1)
    def fit_transform(self, X, y=None):
        onehotencoded = self.fit(X).transform(X)
        return np.delete(onehotencoded, self.last_indexes, axis=1)
    def get_params(self, deep=True):
        return {"features_values" : self.features_values}
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            self.setattr(parameter, value)
        return self

In [4]:
# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv("datasets/dataset_corkA.csv")

In [5]:
# The features we want to select
numeric_features = ["flarea", "bdrms", "bthrms", "floors"]
nominal_features = ["type", "devment", "ber", "location"]

# Create the pipelines
numeric_pipeline = Pipeline([
        ("selector", DataFrameSelector(numeric_features))
    ])

nominal_pipeline = Pipeline([
        ("selector", DataFrameSelector(nominal_features)), 
        ("binarizer", FeatureBinarizer([df[feature].unique() for feature in nominal_features]))])

pipeline = Pipeline([("union", FeatureUnion([("numeric_pipeline", numeric_pipeline), 
                                             ("nominal_pipeline", nominal_pipeline)]))])

In [6]:
# Create the estimator
linreg = LinearRegression()

In [7]:
# Get the target values
y = df["price"].values

In [8]:
# Run the pipeline to prepare the data
pipeline.fit(df)
X = pipeline.transform(df)

In [9]:
# Fit the linear model
linreg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

<h1>How Good Is This Model?</h1>
<ul>
    <li>We've built an estimator by learning a model from a dataset</li>
    <li>We want to know how well it will do in practice, once we start to use it to make predictions
        <ul>
            <li>This is called <b>error estimation</b></li>
        </ul>
    </li>
    <li>Easy right? 
        <ul>
            <li>The dataset comes with <em>actual</em> target values</li>
            <li>We can ask the estimator to <em>predict</em> target values for each example in the dataset</li>
            <li>So now we have actual and predicted values, we can compute the mean squared error</li>
        </ul>
    </li>
</ul>

In [10]:
y_predicted = linreg.predict(X)

In [11]:
mean_squared_error(y, y_predicted)

3924.7640981932445

<ul>
    <li>But, for at least two reasons, we don't do this!
        <ul>
            <li>We might want to use a different performance measure than what we used as the loss function</li>
            <li>We want to know how well the model <b>generalizes</b> to <b>unseen data</b></li>
        </ul>
    </li>
</ul>

<h1>Choosing a Different Performance Measure</h1>
<ul>
    <li>Often in machine learning, we use one measure during learning and another for evaluation</li>
    <li>Class exercise: We already saw this with $k$-means clustering. Explain!</li>
    <li>Our loss function (mean squared error or half of it!) was ideal for learning (why?)
        but may not be so good as a performance measure
        <ul>
            <li>We could use <b>root mean squared error</b> (RMSE):
                $$\sqrt{\frac{1}{m}\sum_{i=1}^m(h_{\v{\beta}}(\v{x}^{(i)}) - \v{y}^{(i)})^2}$$
                (i.e don't halve the MSE, and take its square root: it's the standard deviation
                of the errors in the predictions)
            </li>
            <li>We could use <b>mean absolute error</b> (MAE):
                $$\frac{1}{m}\sum_{i=1}^m\abs(h_{\v{\beta}}(\v{x}^{i)}) - \v{y}^{(i)})$$
            </li>
        </ul>
    </li>
 </ul>

In [12]:
mean_absolute_error(y, y_predicted)

41.638396337007784

<h1>Generalizing to Unseen Data</h1>
<ul>
    <li>The error on the training set is called the <b>training error</b>
        (also 'resubstitution error' and 'in-sample error')
    </li>
    <li>But we want to know how well we will perform in the future, on <em>unseen data</em>
        <ul>
            <li>The training error is not, in general a good indicator of performance on unseen data</li>
            <li>It's often too optimistic. Why?</li>
        </ul>
    </li>
    <li>To predict future performance, we need to measure error on an <em>independent</em> dataset
        <ul>
            <li>A dataset that played no part in creating the estimator</li>
            <li>This second dataset is called the <b>test set</b></li>
            <li>The error on the test set is called the <b>test error</b> (also 'out-of-sample error' and
                'extra-sample error')
            </li>
       </ul>
   </li>
</ul>

<h1>Holdout</h1>
<ul>
    <li>So we use the following method:
       <ul>
           <li><em>Partition</em> our dataset at random into two:
               <ul>
                   <li>training set (e.g. 80% of the full dataset)</li>
                   <li>test set (the rest of the full dataset)</li>
                </ul>
           </li>
           <li>Train the estimator on the training set</li>
           <li>Test the model (evaluate the predictions) on the test set</li>
       </ul>
   </li>
   <li>
       This method is called the <b>holdout</b> method, because the test set
       is withheld (held-out) during training
       <ul>
           <li>It is essential that the test set is not used in any way to create
               the estimator
           </li>
           <li><em>Don't even look at it!</em>
           </li>
           <li>'Cheating' is called <b>leakage</b></li>
           <li>'Cheating' is one cause of <b>overfitting</b> (see <i>CS4619</i>)</li>
       </ul>
    </li>
    <li>Class exercise: Standardization, as we know, is about scaling the data. It requires calculation
        of the mean and standard deviation. When should the mean and standard deviation be calculated:
        (a) before splitting, on the entire dataset, or (b) after splitting, on just the training set?
        Why?
    </li>
</ul>

<h1>Holdout in scikit-learn</h1>

In [13]:
# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv("datasets/dataset_corkA.csv")

In [14]:
# The features we want to select
numeric_features = ["flarea", "bdrms", "bthrms", "floors"]
nominal_features = ["type", "devment", "ber", "location"]

# Create the pipelines
numeric_pipeline = Pipeline([
        ("selector", DataFrameSelector(numeric_features))
    ])

nominal_pipeline = Pipeline([
        ("selector", DataFrameSelector(nominal_features)), 
        ("binarizer", FeatureBinarizer([df[feature].unique() for feature in nominal_features]))])

pipeline = Pipeline([("union", FeatureUnion([("numeric_pipeline", numeric_pipeline), 
                                             ("nominal_pipeline", nominal_pipeline)])),
                     ("estimator", LinearRegression())])

In [15]:
# Get the target values
y = df["price"].values

In [16]:
# Create the object that splits the data
ss = ShuffleSplit(n_splits=1, train_size=0.8)

In [17]:
# Run the pipeline
cross_val_score(pipeline, df, y, scoring="neg_mean_absolute_error", cv=ss)

array([-51.65689577])

<ul>
    <li>This is the negative of the MAE &mdash; so that higher values (closer to zero ) are better</li>
    <li>Compare this value to what we got earlier, when we were training and testing on the whole dataset</li>
    <li>Run it again: what do you notice?</li>
</ul>

<h1>Pipelines Explained</h1>
<ul>
    <li>We are finally in a better position to explain pipelines in scikit-learn</li>
    <li>A scikit-learn pipeline contains a number of steps
    <li>All the steps except the last one must be <b>transformers</b>:
        <ul>
            <li>They are used to transform the data, e.g. to scale it; to binarize it; 
                to reduce its dimensions; &hellip;
            </li>
            <li>They have a method called <code>fit</code>, which computes any values needed to carry
                out the transformation
                <ul>
                    <li>E.g. what does the <code>fit</code> method of <code>StandardScaler</code> compute?
                    </li>
                    <li>E.g. what about <code>MinMaxScaler</code>? <code>PCA</code>?
                    </li>
                </ul>
            </li>
            <li>They have a method called <code>transform</code>, which uses the values computed by 
                <code>fit</code> to modify whatever data is passed to it
                <ul>
                    <li>E.g what does the <code>transform</code> method of <code>StandardScaler</code>
                        do?
                    </li>
                    <li>What about <code>MinMaxScaler</code>? <code>PCA</code>?
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li>The last step in a pipeline can be an <b>estimator</b>:
        <ul>
            <li>They are used to build models from the data and make predictions
                (typically regression and classification)
            </li>
            <li>They have a method called <code>fit</code> which learns the model from the data
                <ul>
                    <li>E.g. what does the <code>fit</code> method of <code>LinearRegression</code> do?
                    </li>
                </ul>
            </li>
            <li>They have a method called <code>predict</code>, which uses the model learned by the
                <code>fit</code> method to make predictions
            </li>
        </ul>
    </li>
    <li>Pipelines themselves have various methods including:
        <ul>
            <li><code>fit</code>: this calls the <code>fit</code> method of the first step, then its
                <code>transform</code> method; then the <code>fit</code> method of the second step,
                then its <code>transform</code> method; and so on; and, eventually, if the last step is
                an estimator, it calls the <code>fit</code> method of the estimator
            </li>
            <li><code>predict</code>: this calls the <code>transform</code> method of the first step;
                then the <code>transform</code> method of the second step; and so on; and, eventually, if the
                last step is an estimator, it calls the <code>predict</code> method of the estimator
            </li>
        </ul>
        Hence it makes sense:
        <ul>
            <li>to call the pipeline's <code>fit</code> method on the <em>training set</em></li>
            <li>then to call the pipeline's <code>predict</code> method on the <em>test set</em></li>
        </ul>
    </li>
</ul>

<h1>Pipelines Explained, continued</h1>
<ul>
    <li>You pass your training set into a pipeline's <code>fit</code> method:
        <figure>
            <img src="images/17_pipeline1.png" />
        </figure>
    </li>
    <li>Then, you pass your test data into a pipeline's <code>predict</code> method: 
        <figure>
            <img src="images/17_pipeline2.png" />
        </figure>
    </li>
</ul>

<h1><code>cross_val_score</code> explained</h1>

In [18]:
# Create the object that splits the data
ss = ShuffleSplit(n_splits=1, train_size=0.8)

# Run the pipeline
cross_val_score(pipeline, df, y, scoring="neg_mean_absolute_error", cv=ss)

array([-59.31289355])

<ul>
    <li><code>cross_val_score</code> takes in the full dataset and the target values</li>
    <li>It splits the dataset in a way determined by its <code>cv</code> parameter</li>
    <li>It calls the pipeline's <code>fit</code> method on the training set</li>
    <li>It calls the pipeline's <code>predict</code> method on the test set</li>
    <li>It compares the test set's actual target values with the test set's predictions using the
        scoring function
    </li>
</ul>

<h1>Pros and Cons of Holdout</h1>
<ul>
    <li>The advantage of holdout is:
        <ul>
            <li>The test error is independent of the training set</li>
        </ul>
    </li>
    <li>The disadvantages of this method are:
        <ul>
            <li>Results can vary quite a lot across different runs
                <ul>
                    <li>Informally, you might get lucky &mdash; or unlucky</li>
                </ul>
                I.e. in any one split, the data used for training or testing might not be representative
            </li>
            <li>We are training on only a subset of the available dataset, perhaps as little as 50% of it
                <ul>
                    <li>From so little data, we may learn a worse model and so our error measurement may 
                        be pessimistic
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li>In practice, we only use the holdout method when we have a very large dataset 
        <ul>
            <li>The size  of the dataset mitigates the above problems</li>
        </ul>
    </li>
    <li>
        When we have a smaller dataset, we use a <b>resampling</b> method:
        <ul>
            <li>The examples get re-used for training and testing</li>
        </ul>
    </li> 
</ul>

<h1>$k$-Fold Cross-Validation</h1>
<ul>
    <li>The most-used resampling method is $k$-fold cross-validation:
        <ul>
            <li>We randomly partition the data into $k$ disjoint subsets of equal size
                <ul>
                    <li>Each of the partitions is called a <b>fold</b></li>
                    <li>Typically, $k = 10$, so you have 10 folds</li>
                    <li>But, for conventional statistical significance testing to be applicable, you should probably ensure 
                        that the number of examples in each fold does not fall below 30. (If this isn't possible, then either 
                        use a smaller value for $k$, or do not use $k$-fold cross validation!)
                    </li>
                </ul>
            </li>
            <li>You take each fold in turn and use it as the test set, training the learner on 
                the remaining folds
            </li>
            <li>Clearly, you can do this $k$ times, so that each fold gets 'a turn' at being the test set
                <ul>
                    <li>
                        By this method, each example is used exactly once for testing, and $k - 1$ times for training
                    </li>
                </ul>
            </li>
        </ul>
    <li>In pseudocode:
        <ul style="background: lightgray; list-style: none">
            <li>
                partition the dataset $D$ into $k$ disjoint equal-sized subsets, $T_1, T_2,\ldots,T_k$
            <li>
            <li>
                <b>for</b> $i = 1$ to $k$
                <ul>
                    <li>train on $D \setminus T_i$</li>
                    <li>make predictions for $T_i$</li>
                    <li>measure error (e.g. MAE)</li>
                </ul>
                report the mean of the errors
            </li>
        </ul>
    </li>
</ul>


<h2>Pros and Cons of $k$-Fold Cross-Validation</h2>
<ul>
    <li>Pros:
        <ul>
            <li>
                The test errors of the folds are independent &mdash; because examples are included in only one test set
            </li>
            <li>
                Better use is made of the dataset: for $k = 10$, for example, we train using 9/10 of the dataset
            </li>
        </ul>
    </li>
    <li>Cons:
        <ul>
            <li>
                While the test sets are independent of each other, the training sets are not: 
                <ul>
                    <li>They will overlap with each other to some degree</li>
                    <li>(This effect of this will be less, of course, for larger datasets)</li>
                </ul>
            </li>
            <li>
                The number of folds is constrained by the size of the dataset and the desire to have folds of
                at least 30 examples
            </li>
            <li>
                It can be costly to train the learning algorithm $k$ times
            </li>
            <li>
                There may still be some variability in the results due to 'lucky'/'unlucky' splits 
            </li>
        </ul>
    </li>
</ul>

<h2>$k$-Fold Cross Validation in scikit-learn</h2>

In [19]:
# Create the object that splits the data
kf = KFold(n_splits = 10)

# Run the pipeline
np.mean(cross_val_score(pipeline, df, y, scoring="neg_mean_absolute_error", cv=kf))

-59.428212082117433

<ul>
    <li>But $k$-fold cross-validation is so common, there's a shorthand:</li>
</ul>   

In [20]:
np.mean(cross_val_score(pipeline, df, y, scoring="neg_mean_absolute_error", cv=10))

-59.428212082117433

<ul>
    <li>Be warned, however, this almost certainly does not shuffle the dataset before splitting it into folds
        <ul>
            <li>Why might that be a problem?</li>
        </ul>
    </li>
    <li>
        You should probably shuffle the <code>DataFrame</code> just after reading it in from the CSV file 
        (see example below)
    </li>
</ul>

<h1>Final Remarks</h1>
<ul>
    <li>
        There are many resampling methods other than $k$-Fold Cross-Validation:
        <ul>
            <li>Repeated $k$-Fold Cross-Validation, Leave-One-Out-Cross-Validation, 
                &hellip;
            </li>
        </ul>
    </li>
    <li>
        So you've used one of the above methods and found the test error of your estimator.
        <ul>
            <li>This is supposed to give you an idea of how your estimator will perform in practice</li>
            <li>What if you are dissatisfied with the test error? It seems too high
                <ul>
                    <li>It is tempting to tweak your learning algorithm or try different algorithms
                        to try to bring down the test error
                    </li>
                    <li>This is wrong! It is <b>leakage</b> again: you will be using knowledge of the test 
                        set to develop the estimator and is likely to result in an optimistic view of the 
                        ultimate performance of the estimator on unseen data
                    </li>
                    <li>Ideally, error estimation on the test set is the last thing you do
                </ul>
            </li>
        </ul>
    </li>
    <li>
        Finally, suppose you have used one of the above methods to estimate the error of your regressor. 
        You are ready to release your regressor on the world. At this point, you can train it on
        <em>all</em> the examples in your dataset, so as to maximize the use of the data
    </li>
</ul>

<h1>A Little Case Study in scikit-learn</h1>

In [21]:
# Use pandas to read the CSV file into a DataFrame
df = pd.read_csv("datasets/dataset_corkA.csv")

In [22]:
# Shuffle
df = df.take(np.random.permutation(len(df)))

In [23]:
# The features we want to select
numeric_features = ["flarea", "bdrms", "bthrms", "floors"]
nominal_features = ["type", "devment", "ber", "location"]

# Create the pipelines
numeric_pipeline = Pipeline([
        ("selector", DataFrameSelector(numeric_features)),
    ])

numeric_pipeline_with_PCA = Pipeline([
        ("selector", DataFrameSelector(numeric_features)),
        ("pca", PCA(n_components=0.9))
    ])

nominal_pipeline = Pipeline([
        ("selector", DataFrameSelector(nominal_features)), 
        ("binarizer", FeatureBinarizer([df[feature].unique() for feature in nominal_features]))])

pipeline = Pipeline([("union", FeatureUnion([("numeric_pipeline", numeric_pipeline), 
                                             ("nominal_pipeline", nominal_pipeline)])),
                         ("estimator", LinearRegression())])

pipeline_with_PCA = Pipeline([("union", FeatureUnion([("numeric_pipeline", numeric_pipeline_with_PCA), 
                                                      ("nominal_pipeline", nominal_pipeline)])),
                              ("estimator", LinearRegression())])

In [24]:
# Get the target values
y = df["price"].values

In [25]:
# Run the no-PCA pipeline
np.mean(cross_val_score(pipeline, df, y, scoring="neg_mean_absolute_error", cv=10))

-57.493916350809968

In [26]:
# Run the pipeline with PCA
np.mean(cross_val_score(pipeline_with_PCA, df, y, scoring="neg_mean_absolute_error", cv=10))

-57.457440456251007

<ul>
    <li>Final observation: In the above, we ran 10-fold cross validation on the Cork property dataset but it has 
        only 224 examples &mdash; not enough examples to give at least 30 examples in each of the 10 folds
    </li>
    <li>So this isn't an ideal use of the method</li>
</ul>